# Sections
## - Load all the pre-processed user review text data
## - Group "Seen" users text data correctly
## - Test "construct_user_vector" function
## - Apply "construct_user_vector" and store for all users

In [1]:
import numpy
import pandas

## Load all the pre-processed user review text data

In [3]:
ts1=pandas.read_pickle('data/rec_task_train_set1.pkl')
ts2=pandas.read_pickle('data/rec_task_train_set2.pkl')
ts3=pandas.read_pickle('data/rec_task_train_set3.pkl')
unseen=pandas.read_pickle('data/rec_task_test_unseen_users.pkl')
seen=pandas.read_pickle('data/rec_task_test_seen_users.pkl')

In [4]:
print(ts1.columns)
print(ts2.columns)
print(ts3.columns)
print(unseen.columns)
print(seen.columns)

Index(['user_id', 'train_data_text'], dtype='object')
Index(['user_id', 'train_data_text'], dtype='object')
Index(['user_id', 'train_data_text'], dtype='object')
Index(['review_id', 'user_id', 'business_id', 'review_stars', 'useful',
       'funny', 'cool', 'date', 'name', 'city', 'business_stars',
       'review_count', 'is_open', 'categories', 'text_prep'],
      dtype='object')
Index(['review_id', 'user_id', 'business_id', 'review_stars', 'useful',
       'funny', 'cool', 'date', 'name', 'city', 'business_stars',
       'review_count', 'is_open', 'categories', 'text_prep'],
      dtype='object')


In [6]:
unseen['user_text']=unseen.groupby(['user_id'])['text_prep'].transform(lambda x: '\t'.join(x))
unseen_sub=unseen.drop(['review_id','business_id','review_stars','useful','funny','cool',
                                'date','name','city','business_stars','review_count','is_open',
                                'categories','text_prep'],axis=1)
unseen_sub.drop_duplicates(inplace=True)
print(unseen_sub.shape)
print(unseen_sub.columns)

(25040, 2)
Index(['user_id', 'user_text'], dtype='object')


In [7]:
seen['user_text']=seen.groupby(['user_id'])['text_prep'].transform(lambda x: '\t'.join(x))
seen_sub=seen.drop(['review_id','business_id','review_stars','useful','funny','cool',
                                'date','name','city','business_stars','review_count','is_open',
                                'categories','text_prep'],axis=1)
seen_sub.drop_duplicates(inplace=True)
print(seen_sub.shape)
print(seen_sub.columns)

(17017, 2)
Index(['user_id', 'user_text'], dtype='object')


In [10]:
ts1=ts1.rename({'train_data_text':'user_text'},axis=1)
ts2=ts2.rename({'train_data_text':'user_text'},axis=1)
ts3=ts3.rename({'train_data_text':'user_text'},axis=1)

In [12]:
print(ts3.shape)

(9614, 2)


In [13]:
print(seen_sub['user_id'].nunique())

17017


### Group "Seen" users text data correctly

In [11]:
combine_seen_train_test=pandas.concat([seen_sub,ts3],ignore_index=True)
print(combine_seen_train_test.shape)
print(combine_seen_train_test.columns)

(26631, 2)
Index(['user_id', 'user_text'], dtype='object')


In [14]:
combine_seen_train_test['user_text_final']=combine_seen_train_test.groupby(['user_id'])['user_text'].transform(lambda x: '\t'.join(x))

In [16]:
combine_seen_train_test=combine_seen_train_test.drop(['user_text'],axis=1)
combine_seen_train_test.drop_duplicates(inplace=True)

In [17]:
print(combine_seen_train_test.shape)
print(combine_seen_train_test.columns)

(17017, 2)
Index(['user_id', 'user_text_final'], dtype='object')


In [18]:
combine_seen_train_test=combine_seen_train_test.rename({'user_text_final':'user_text'},axis=1)

In [19]:
user_trove_text=pandas.concat([ts1,ts2,unseen_sub,combine_seen_train_test],ignore_index=True)
print(user_trove_text.shape)
print(user_trove_text.columns)

(1251983, 2)
Index(['user_id', 'user_text'], dtype='object')


In [20]:
print(user_trove_text.iloc[1251980]['user_text'])

super friendly staff came lunch bffs 10 30am open early bonus great greasy reuben girlfriend ended eating burger knife fork ate happily generous amount fry favorite sure describe would go chip next time generally love fry place whole cooler root beer different brand course 2 additional cooler every flavor soda could think including retro soda like bubble overall nice little place seems get good traffic even 10 30 location needing lunch place like long time health food good back gotta load napkin	tried place reading great review yelp glad really enjoyed couple beer outdoor patio beer excellent sunday morning saison midnight run coffee stout husband rating 4 star instead 5 due lackluster service bad friendly calm sunday afternoon overall definitely back look like fun place enjoy couple beer especially weather good sit outside open hopefully service bit welcoming next time regardless beer worth coming back	happened place way 12 west new taproom gilbert stopped see like incredibly friendly

In [ ]:
dims=50
dimheader=[]
for i in range(dims):
    dimheader.append(str(i))
print(dimheader)

In [25]:
#load embeddings
embeddingsdf=pandas.read_table('models/embeddings/model.tsv',header=None,names=['word']+dimheader)
print(embeddingsdf.shape)
#print(embeddingsdf.iloc[5])

(237728, 51)


In [26]:
onlyembeddings=embeddingsdf.drop(['word'],axis=1).to_numpy()

In [29]:
print(onlyembeddings.shape)

(237728, 50)


In [31]:
word2index={}
index2word={}
for i in range(embeddingsdf.shape[0]):
    word=embeddingsdf.iloc[i]['word']
    word2index[word]=i
    index2word[i]=word

In [33]:
print(word2index['amazing'])
print(index2word[20])

20
amazing


In [56]:
def construct_user_vector(ut):
    utsents=ut.split('\t')
    utwords=[]
    for sent in utsents:
        words=sent.split(' ')
        utwords+=words
    utids=[]
    for word in utwords:
        try:
            wi=word2index[word]
        except:
            continue
        utids.append(wi)
    #print(len(utids))
    embeddingmatrix=onlyembeddings[[utids],:]
    #print(embeddingmatrix.shape)
    #print(embeddingmatrix)
    embeddingmatrix=numpy.reshape(embeddingmatrix,(len(utids),onlyembeddings.shape[1]))
    #print(embeddingmatrix.shape)
    user_embedding=numpy.mean(embeddingmatrix,axis=0)
    #print(user_embedding.shape)
    #print(user_embedding)
    return(user_embedding)

## Test "construct_user_vector" function

In [52]:
#construct_user_vector('great service\tgood food')
construct_user_vector('great place\tgood food')

4
(1, 4, 50)
(4, 50)
(50,)
[-0.03114441 -0.00558786 -0.03650153 -0.00589928  0.00994047 -0.00601946
 -0.01408828  0.01285871  0.0348892  -0.01410735 -0.0055935  -0.01012005
  0.02174653  0.01608618 -0.02453382  0.01585097  0.00308508 -0.0256366
 -0.00793062 -0.02370774  0.00675338  0.02222649 -0.00158792 -0.0127453
  0.01447883  0.05925738  0.0299479   0.00798461 -0.02519177 -0.00645107
  0.029131   -0.02759337 -0.02670989  0.00018515  0.01793671  0.00832105
 -0.00974767  0.03323479  0.00374375 -0.00265308  0.0183133  -0.0033772
  0.01664072  0.00422745 -0.00924635  0.01644893 -0.016141    0.0207536
 -0.03818242  0.02055518]


array([-0.03114441, -0.00558786, -0.03650153, -0.00589928,  0.00994047,
       -0.00601946, -0.01408828,  0.01285871,  0.0348892 , -0.01410735,
       -0.0055935 , -0.01012005,  0.02174653,  0.01608618, -0.02453382,
        0.01585097,  0.00308508, -0.0256366 , -0.00793062, -0.02370774,
        0.00675338,  0.02222649, -0.00158792, -0.0127453 ,  0.01447883,
        0.05925738,  0.0299479 ,  0.00798461, -0.02519177, -0.00645107,
        0.029131  , -0.02759337, -0.02670989,  0.00018515,  0.01793671,
        0.00832105, -0.00974767,  0.03323479,  0.00374375, -0.00265308,
        0.0183133 , -0.0033772 ,  0.01664072,  0.00422745, -0.00924635,
        0.01644893, -0.016141  ,  0.0207536 , -0.03818242,  0.02055518])

In [61]:
construct_user_vector('great great place')

array([-0.05138747, -0.00163976, -0.0701213 , -0.03429977, -0.0056659 ,
       -0.01026472, -0.00516783,  0.01041776,  0.06018643, -0.00222773,
       -0.02343767, -0.02159223,  0.0356687 ,  0.05788273,  0.0027088 ,
        0.046091  , -0.01140145, -0.00484977, -0.0101957 , -0.02494785,
        0.01355051,  0.00145659, -0.0037733 , -0.05955048,  0.0344942 ,
        0.05056943,  0.0217402 , -0.02027992, -0.00142567,  0.00199353,
        0.07822237,  0.02446197, -0.00641833, -0.00265917,  0.01999543,
        0.02482507,  0.0021395 ,  0.04680101,  0.01987697, -0.02204873,
        0.0180526 ,  0.0004619 , -0.0023353 , -0.00099829, -0.04831893,
       -0.0161306 , -0.03972167,  0.0197076 , -0.05024077,  0.01819029])

In [45]:
uttsub=user_trove_text[:5]
print(uttsub)
uttsub['user_vector']=uttsub['user_text'].apply(construct_user_vector)

                  user_id                                          user_text
0  yXQM5uF2jS6es16SJzNHfg  adore travis hard rock new kelly cardenas salo...
1  n6-Gk65cPZL6Uz8qRm3NYw  say office really together organized friendly ...
2  dacAIZ6fTM6mqwW5uxkskg  went lunch steak sandwich delicious caesar sal...
3  w31MKYsNFMrjhWxxAb5wIw  first admit excited going la tavolta food snob...
4  FIk4lQQu1eTe2EpzQ4xhBA  like walking back time every saturday morning ...


In [55]:
print(uttsub.iloc[0]['user_vector'])

[ 2.20885017e-03 -2.76133208e-03 -1.71596288e-03  4.10782680e-05
  3.02524725e-03 -1.25246633e-02 -1.85634095e-02 -6.91440986e-03
 -1.11139981e-02  5.14055163e-03  2.67222332e-03  1.64532948e-02
  6.51711419e-03 -2.99987846e-04 -1.04801459e-02  1.06025856e-02
  7.30883853e-03  2.00066369e-02  2.78504871e-03  4.10778843e-03
  3.79805388e-03  1.02089387e-02  7.52436733e-04  1.42835346e-02
  1.20174594e-03 -1.45579899e-03 -4.82707168e-03 -3.82220286e-03
  3.61898196e-03 -9.64714905e-03 -2.71067079e-03  1.18350605e-02
  3.45518855e-03  2.30898850e-03  1.69463082e-03 -3.92159342e-03
  1.82942177e-02 -8.58568314e-03  2.45915010e-03 -1.28370141e-02
 -3.98532768e-03 -6.36206386e-03 -5.67608980e-03  5.69927338e-03
 -1.86201813e-03  1.86407907e-02  3.14488727e-03 -4.32414319e-03
  9.82206915e-03  4.93275912e-03]


## Apply "construct_user_vector" and store for all users

In [57]:
%%time
user_trove_text['user_vector']=user_trove_text['user_text'].apply(construct_user_vector)
print(user_trove_text.shape)
print(user_trove_text.columns)

(1251983, 3)
Index(['user_id', 'user_text', 'user_vector'], dtype='object')
CPU times: user 1min 54s, sys: 4.75 s, total: 1min 59s
Wall time: 1min 59s


In [58]:
user_trove=user_trove_text.drop(['user_text'],axis=1)
print(user_trove.shape)
print(user_trove.columns)

(1251983, 2)
Index(['user_id', 'user_vector'], dtype='object')


In [60]:
user_trove.to_pickle('models/user_trove.pkl')